In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
from torch import nn
import time
import gc

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B").to('cuda')
question="hi"
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
# prompt=tok.apply_chat_template(converstation)
prompt=tok(question,return_tensors='pt').to('cuda')
out=model.generate(**prompt,max_new_tokens=10)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [4]:
out=model.generate(**prompt,max_new_tokens=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [3]:
out[0]

tensor([128000,   6151,   1268,    527,    499,     30,    602,    574,  20910,
           422,    499,   1436,   1520,    757,    449,   2555,     11,    602,
           617,    264,    220,   2550,     22,    220,     19],
       device='cuda:0')

In [5]:
# If out is a tensor (e.g., shape [1, seq_len])
decoded = tok.decode(out[0], skip_special_tokens=True)
print(decoded)


hi, I have a problem with my 3D


In [2]:
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B").to('cpu')
print(model)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [10]:
model.model.rotary_emb

LlamaRotaryEmbedding()

In [11]:
for i in model.model.layers:
    print(i)
    break

LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear(in_features=2048, out_features=512, bias=False)
    (v_proj): Linear(in_features=2048, out_features=512, bias=False)
    (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)


In [34]:
dtype = torch.bfloat16
bytes_per_param = torch.tensor([], dtype=dtype).element_size()
bytes_per_param

2

In [40]:
l=model.model.layers[0]
num_params = sum(p.numel() for p in l.parameters())
num_params
param_size_bytes = num_params * 4
param_size_MB = param_size_bytes / (1024 ** 3)
param_size_MB 

0.2265777587890625

In [38]:
(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)

7.6064453125

In [42]:
int((torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)/param_size_MB )

33

In [45]:
layers=model.model.layers
len(layers)

16

In [ ]:
layers=layers[:-1]
len(layers)

In [3]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 4
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

In [4]:
get_layers(model)

31

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch import nn
import time
import gc

# Enable CUDA and Tensor Core optimizations
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

# Load the model and tokenizer
# Corrected the missing closing quote in the tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                             torch_dtype=torch.bfloat16,
                                             use_cache=True,
                                             # quantization_config=bnb_config
                                             )
model.to('cpu')

tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")


def get_layers(model, drop=2):
    """
    Calculates the maximum number of layers that can fit on the GPU.
    This function has been improved to account for static model parts
    and provide a more robust chunk size calculation.

    Args:
        model: The model instance.
        drop (int): A memory buffer in GB to leave on the GPU.

    Returns:
        int: The number of layers per chunk.
    """
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024**3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2 # Using bfloat16 for correct size calculation
    param_size_GB = param_size_bytes / (1024 ** 3)
    
    # Adjusting for static model parts which are loaded once
    static_mem_gb = 0
    if hasattr(model.model, 'embed_tokens'):
        static_mem_gb += sum(p.numel() for p in model.model.embed_tokens.parameters()) * 2 / (1024**3)
    if hasattr(model.model, 'norm'):
        static_mem_gb += sum(p.numel() for p in model.model.norm.parameters()) * 2 / (1024**3)
    if hasattr(model, 'lm_head'):
        static_mem_gb += sum(p.numel() for p in model.lm_head.parameters()) * 2 / (1024**3)

    available_gpu_memory_for_layers_gb = device_memory - drop - static_mem_gb
    if available_gpu_memory_for_layers_gb <= 0:
        print(f"Warning: Not enough GPU memory. Available: {device_memory:.2f} GB. Falling back to 1 layer per chunk.")
        return 1
    
    if param_size_GB == 0:
        return 1

    return max(1, int(available_gpu_memory_for_layers_gb / param_size_GB))


class Module(nn.Module):
    """A wrapper for a chunk of layers."""
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
    
    @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            # Pass the attention mask to the layer
            x = layer(x, position_embeddings=(cos, sin), attention_mask=attention_mask)[0]
        return x


def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    """
    Computes a forward pass using a three-worker pipeline with three explicit streams.
    Worker 1: Computation
    Worker 2: Loading new layers
    Worker 3: Offloading old layers
    """
    total_layers = len(model_instance.model.layers)

    stream_compute = torch.cuda.Stream()
    stream_load = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    # Case 1: All layers fit on the GPU, no offloading needed.
    if layers_per_chunk >= total_layers:
        m = Module(model_instance.model.layers, 0, total_layers).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m.to('cpu')
            torch.cuda.empty_cache()
            del m
        return x

    # Case 2: Pipelined offloading with three streams
    i = 0
    rem = total_layers
    
    # Priming the pipeline: Load the first chunk on the load stream
    m_current = None
    with torch.cuda.stream(stream_load):
        m_current = Module(model_instance.model.layers, i, i + layers_per_chunk).to('cuda', non_blocking=True)
    i += layers_per_chunk
    rem -= layers_per_chunk

    m_next = None
    
    while True:
        # Load the next chunk if there are layers remaining
        if rem > 0:
            l_next = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_load):
                m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)
        else:
            m_next = None

        # Wait for the current chunk to finish loading
        stream_compute.wait_stream(stream_load)

        # Compute on the current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x, cos, sin, attention_mask)

        # Check if this is the last chunk
        if rem <= 0:
            break
        
        # Offload the current chunk asynchronously
        with torch.cuda.stream(stream_offload):
            m_current.to('cpu', non_blocking=True)

        # Advance the pipeline and update counters
        m_current = m_next
        i += l_next
        rem -= l_next

    # Final cleanup after the loop
    stream_compute.synchronize()
    # The last m_current must be moved back to cpu explicitly
    m_current.to('cpu')
    del m_current
    if m_next:
        del m_next
    torch.cuda.empty_cache()
    return x


def model_inference(question, tokens=250, temperature=0.7, top_p=0.95):
    start_total_time = time.time()
    
    model.to('cpu')
    
    prompt = tok(question, return_tensors='pt').to('cpu')
    input_ids = prompt['input_ids'].to('cuda')
    # Correctly move attention mask to CUDA and convert its dtype
    attention_mask = prompt['attention_mask'].to('cuda')
    del prompt

    # Move static parts of the model to GPU once
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    rotary_emb = model.model.rotary_emb
    
    generated_text_tokens = [] 

    for k in range(tokens):
        x = model.model.embed_tokens(input_ids)
        
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        cos, sin = rotary_emb(x=x,position_ids=position_ids)

        # Create a 4D attention mask with the correct dtype
        causal_mask = torch.triu(torch.ones((1, seq_len, seq_len), device=x.device, dtype=torch.bool), diagonal=1)
        causal_mask = ~causal_mask
        causal_mask = causal_mask.view(1, 1, seq_len, seq_len)
        causal_mask = causal_mask * attention_mask.view(1, 1, 1, seq_len)
        causal_mask = (causal_mask == 0).float() * torch.finfo(x.dtype).min
        
        with torch.no_grad():
            l1 = get_layers(model, 7)
            # Use the calculated l1 value, not a hardcoded number
            x = compute(model, x, l1, cos, sin, k, tokens, causal_mask)
            
            x = model.model.norm(x)
            x = model.lm_head(x)
        
        # Apply temperature and Top-P sampling
        logits = x[:, -1, :] / temperature
        
        # Sort logits in descending order
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        
        # Calculate cumulative probabilities
        cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
        
        # Find the tokens that are in the nucleus
        sorted_indices_to_remove = cumulative_probs > top_p
        
        # Shift the indices to the right to keep at least one token
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        
        # Mask out the logits that are not in the nucleus
        logits[sorted_indices[sorted_indices_to_remove]] = -float('inf')
        
        # Sample a token from the nucleus
        next_token = torch.multinomial(torch.softmax(logits, dim=-1), num_samples=1)
        
        print(tok.decode(next_token[0]), end="")
        generated_text_tokens.append(tok.decode(next_token[0]))
        
        input_ids = torch.cat([input_ids, next_token.to('cuda')], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(attention_mask.shape[0], 1).to('cuda')], dim=-1)
    
    # Final cleanup at the very end
    model.model.embed_tokens.to('cpu')
    model.model.norm.to('cpu')
    model.lm_head.to('cpu')
    torch.cuda.empty_cache() 
    
    end_total_time = time.time()
    print(f"\nTime taken for {tokens} tokens: {end_total_time - start_total_time:.2f} seconds")
    return "".join(generated_text_tokens)


# Example usage
print("Starting inference...")
start = time.time()
model_inference("HI", 10)
print()
print("Total time", time.time() - start)


/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting inference...


W0804 17:45:32.001540 23607 site-packages/torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:2177: UserWarning: NVIDIA T1000 8GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/pyt

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch import nn
import time
import gc

# Enable CUDA and Tensor Core optimizations
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

# Load the model and tokenizer
# Corrected the missing closing quote in the tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                             torch_dtype=torch.bfloat16,
                                             use_cache=True,
                                             # quantization_config=bnb_config
                                             )
model.to('cpu')

tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc
# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           use_cache=True,
                                        #    quantization_config=bnb_config
                                           )
# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:



def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

class Module(nn.Module):
    """A wrapper for a chunk of layers."""
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
    
    @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            # Pass the attention mask to the layer
            x = layer(x, position_embeddings=(cos, sin), attention_mask=attention_mask)[0]
        return x


def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    """
    Computes a forward pass using a three-worker pipeline with three explicit streams.
    Worker 1: Computation
    Worker 2: Loading new layers
    Worker 3: Offloading old layers
    """
    total_layers = len(model_instance.model.layers)

    stream_compute = torch.cuda.Stream()
    stream_load = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    # Case 1: All layers fit on the GPU, no offloading needed.
    if layers_per_chunk >= total_layers:
        m = Module(model_instance.model.layers, 0, total_layers).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m.to('cpu')
            torch.cuda.empty_cache()
            del m
        return x

    # Case 2: Pipelined offloading with a robust, explicit three-stage loop.
    i = 0
    
    # Priming the pipeline: Load the first chunk
    m_current = None
    with torch.cuda.stream(stream_load):
        m_current = Module(model_instance.model.layers, i, i + layers_per_chunk).to('cuda', non_blocking=True)
    i += layers_per_chunk
    
    # Initialize offload module to None for the first iteration
    m_to_offload = None
    
    while i <= total_layers:
        # Load the next chunk if there are layers remaining
        m_next = None
        if i < total_layers:
            l_next = min(layers_per_chunk, total_layers - i)
            with torch.cuda.stream(stream_load):
                m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)

        # Wait for the current chunk to finish loading before computing
        stream_compute.wait_stream(stream_load)
        
        # Compute on the current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x, cos, sin, attention_mask)
        
        # Offload the previously used chunk (if it exists)
        if m_to_offload:
            # Wait for compute to finish before offloading to prevent race condition
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_to_offload.to('cpu', non_blocking=True)

        # Advance the pipeline
        m_to_offload = m_current
        m_current = m_next
        i += layers_per_chunk

    # Handle the final cleanup outside the loop
    stream_compute.synchronize()
    stream_load.synchronize()
    stream_offload.synchronize()
    
    # Ensure the last computed module is offloaded as well
    if m_to_offload:
        m_to_offload.to('cpu')
    if m_current: # Should be None but for safety
        m_current.to('cpu')
    if m_next:
        m_next.to('cpu')
    
    del m_to_offload, m_current, m_next
    
    torch.cuda.empty_cache()
    return x


def model_inference(question, tokens=250):
    start_total_time = time.time()
    
    model.to('cpu')
    
    prompt = tok(question, return_tensors='pt').to('cpu')
    input_ids = prompt['input_ids'].to('cuda')
    # Correctly move attention mask to CUDA and convert its dtype
    attention_mask = prompt['attention_mask'].to('cuda')
    del prompt

    # Move static parts of the model to GPU once
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    rotary_emb = model.model.rotary_emb
    
    generated_text_tokens = [] 

    for k in range(tokens):
        x = model.model.embed_tokens(input_ids)
        
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        cos, sin = rotary_emb(x=x,position_ids=position_ids)

        # Create a 4D attention mask with the correct dtype
        causal_mask = torch.triu(torch.ones((1, seq_len, seq_len), device=x.device, dtype=torch.bool), diagonal=1)
        causal_mask = ~causal_mask
        causal_mask = causal_mask.view(1, 1, seq_len, seq_len)
        causal_mask = causal_mask * attention_mask.view(1, 1, 1, seq_len)
        causal_mask = (causal_mask == 0).float() * torch.finfo(x.dtype).min
        
        with torch.no_grad():
            # Forcing layers_per_chunk to 5 as requested to test pipelining
            layers_per_chunk = 5
            x = compute(model, x, layers_per_chunk, cos, sin, k, tokens, causal_mask)
            
            x = model.model.norm(x)
            x = model.lm_head(x)
        
        # Reverting to the original greedy decoding using argmax as requested
        x1 = x[:,-1,:]
        next_token = torch.argmax(x1, dim=-1, keepdim=True)
        
        print(tok.decode(next_token[0]), end="")
        generated_text_tokens.append(tok.decode(next_token[0]))
        
        input_ids = torch.cat([input_ids, next_token.to('cuda')], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(attention_mask.shape[0], 1).to('cuda')], dim=-1)
    
    # Final cleanup at the very end
    model.model.embed_tokens.to('cpu')
    model.model.norm.to('cpu')
    model.lm_head.to('cpu')
    torch.cuda.empty_cache() 
    
    end_total_time = time.time()
    print(f"\nTime taken for {tokens} tokens: {end_total_time - start_total_time:.2f} seconds")
    return "".join(generated_text_tokens)

In [7]:
print("Starting inference...")
start = time.time()
model_inference("HI", 10)
print()
print("Total time", time.time() - start)

Starting inference...


/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:2177: UserWarning: NVIDIA T1000 8GB does not support bfloat16 compilation natively, skipping
  warnings.warn(


centerslyphlyphlyphlyphlyphlyphlyphlyphlyph
Time taken for 10 tokens: 9.55 seconds

Total time 9.552647829055786


# 2 workers

In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc
# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           use_cache=True,
                                        #    quantization_config=bnb_config
                                           )
# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

In [3]:
get_layers(model)


65

In [4]:
print(len(model.model.layers))

16


In [5]:
class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
        
    @torch.compile
    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x
layers=model.model.layers


# Usage
# compiled_block = torch.compile(FusedBlock(layers[i:j]))

# def compute(model, x, layers_per_chunk, cos, sin,k,max_new_tokens,attention_mask):
#     total_layers = len(model.model.layers)
#     gpu_stream = torch.cuda.current_stream()
#     if layers_per_chunk >= total_layers:
#         # model = model.to('cuda')
#         # out = model({'inputs_embeds': x})['logits'][:, -1, :]
#         # return out.argmax(-1)
#         i=0
#         l=total_layers
#         m=Module(model.model.layers,i,i+l).to('cuda')
#         x=m(x.to('cuda', non_blocking=True),cos.to('cuda',non_blocking=True),sin.to('cuda',non_blocking=True),attention_mask.to('cuda'))
#         if(k==max_new_tokens-1):
#             m=m.to('cpu')
#             torch.cuda.empty_cache()
#         return x
#     i = 0
#     rem = total_layers
#     while rem > 0:
#         l = min(layers_per_chunk, rem)
        
#         m = Module(model.model.layers, i, i + l).to('cuda',non_blocking=True)
#         # torch.cuda.current_stream().wait_stream(gpu_stream)
#         # m=torch.compile(m)
#         # Run chunk
#         with torch.no_grad():
#             x=m(x.to('cuda', non_blocking=True),cos.to('cuda',non_blocking=True),sin.to('cuda',non_blocking=True),attention_mask.to('cuda'))
#         if(k==max_new_tokens-1):
#             m=m.to('cpu')
#             torch.cuda.empty_cache()
#         m=m.to('cpu')
#         torch.cuda.empty_cache()
#         # update indices
#         i += l
#         rem -= l
#     torch.cuda.current_stream().synchronize()
#     return x

In [6]:
def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu')
            torch.cuda.empty_cache()
        return x
    i=0
    rem = total_layers
    m_current = None
    m_next = None
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first

    while True:
        stream_compute.wait_stream(stream_transfer)
        m_current = m_next
        m_next = None
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x.to('cuda', non_blocking=True), 
                              cos.to('cuda', non_blocking=True), 
                              sin.to('cuda', non_blocking=True), 
                              attention_mask.to('cuda', non_blocking=True))
        if rem == 0:
            stream_compute.synchronize()
            m_current = m_current.to('cpu')
            torch.cuda.empty_cache()
            break
        with torch.cuda.stream(stream_transfer):
            m_current = m_current.to('cpu', non_blocking=True)
            del m_current
            l_next = min(layers_per_chunk, rem)
            m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)
        torch.cuda.empty_cache() 
        i += l_next
        rem -= l_next
    return x

In [7]:
def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda') 
    model.lm_head.to('cuda')
    # x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb
    
    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cuda')
        # if i!=0:
        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
        
        cnt=0
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,1,cos,sin,k,tokens,attention_mask)
            # model.model.norm.to('cuda')
            x=model.model.norm(x)
            # model.model.norm.to('cpu')
            # torch.cuda.empty_cache()
            # model.lm_head.to('cuda')
            x=model.lm_head(x)
            # model.lm_head.to('cpu')
            # torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1) 
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
        

In [13]:
start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

, I am a 19 year old girl who
Total time 4.548299551010132


# other

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch import nn
import time
import gc

# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                             torch_dtype=torch.bfloat16,
                                             use_cache=True,
                                             )
model.to('cpu')

tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")


/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


def get_layers(model, drop=2):
    """
    Calculates the maximum number of layers that can fit on the GPU.
    This function has been improved to account for static model parts
    and provide a more robust chunk size calculation.

    Args:
        model: The model instance.
        drop (int): A memory buffer in GB to leave on the GPU.

    Returns:
        int: The number of layers per chunk.
    """
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024**3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2 # Using bfloat16 for correct size calculation
    param_size_GB = param_size_bytes / (1024 ** 3)
    
    # Adjusting for static model parts which are loaded once
    static_mem_gb = 0
    if hasattr(model.model, 'embed_tokens'):
        static_mem_gb += sum(p.numel() for p in model.model.embed_tokens.parameters()) * 2 / (1024**3)
    if hasattr(model.model, 'norm'):
        static_mem_gb += sum(p.numel() for p in model.model.norm.parameters()) * 2 / (1024**3)
    if hasattr(model, 'lm_head'):
        static_mem_gb += sum(p.numel() for p in model.lm_head.parameters()) * 2 / (1024**3)

    available_gpu_memory_for_layers_gb = device_memory - drop - static_mem_gb
    if available_gpu_memory_for_layers_gb <= 0:
        print(f"Warning: Not enough GPU memory. Available: {device_memory:.2f} GB. Falling back to 1 layer per chunk.")
        return 1
    
    if param_size_GB == 0:
        return 1

    return max(1, int(available_gpu_memory_for_layers_gb / param_size_GB))


class Module(nn.Module):
    """A wrapper for a chunk of layers."""
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
    
    @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            # Pass the attention mask to the layer
            x = layer(x, position_embeddings=(cos, sin), attention_mask=attention_mask)[0]
        return x


def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    """
    Computes a forward pass using a three-worker pipeline with three explicit streams.
    Worker 1: Loading new layers (stream_load)
    Worker 2: Computation (stream_compute)
    Worker 3: Offloading old layers (stream_offload)
    """
    total_layers = len(model_instance.model.layers)

    stream_compute = torch.cuda.Stream()
    stream_load = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    # Case 1: All layers fit on the GPU, no offloading needed.
    if layers_per_chunk >= total_layers:
        m = Module(model_instance.model.layers, 0, total_layers).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m.to('cpu')
            torch.cuda.empty_cache()
            del m
        return x

    # Case 2: Pipelined offloading with a robust, explicit three-stage loop.
    i = 0
    m_to_offload = None # This will hold the chunk from the previous iteration to be offloaded
    m_current = None # This is the chunk currently being computed
    
    # Priming the pipeline: Load the first chunk
    with torch.cuda.stream(stream_load):
        m_current = Module(model_instance.model.layers, i, i + layers_per_chunk).to('cuda', non_blocking=True)
    i += layers_per_chunk
    
    while i <= total_layers + layers_per_chunk: # Loop until all layers are processed and the last chunk is offloaded
        # 1. Load the next chunk on a separate stream if there are layers left
        m_next = None
        if i < total_layers:
            l_next = min(layers_per_chunk, total_layers - i)
            with torch.cuda.stream(stream_load):
                m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)
        
        # 2. Wait for the current chunk to finish loading, then compute
        stream_compute.wait_stream(stream_load)
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x, cos, sin, attention_mask)
        
        # 3. Wait for the computation to finish, then offload the previous chunk
        if m_to_offload:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_to_offload.to('cpu', non_blocking=True)
        
        # Advance the pipeline to the next stage
        m_to_offload = m_current
        m_current = m_next
        i += layers_per_chunk

    # Final cleanup after the loop
    stream_compute.synchronize()
    stream_load.synchronize()
    stream_offload.synchronize()
    
    del m_to_offload, m_current, m_next
    
    torch.cuda.empty_cache()
    return x


def model_inference(question, tokens=250):
    start_total_time = time.time()
    
    model.to('cpu')
    
    prompt = tok(question, return_tensors='pt').to('cpu')
    input_ids = prompt['input_ids'].to('cuda')
    # Correctly move attention mask to CUDA and convert its dtype
    attention_mask = prompt['attention_mask'].to('cuda')
    del prompt

    # Move static parts of the model to GPU once
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    rotary_emb = model.model.rotary_emb
    
    generated_text_tokens = [] 

    for k in range(tokens):
        x = model.model.embed_tokens(input_ids)
        
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        cos, sin = rotary_emb(x=x,position_ids=position_ids)

        # Create a 4D attention mask with the correct dtype
        causal_mask = torch.triu(torch.ones((1, seq_len, seq_len), device=x.device, dtype=torch.bool), diagonal=1)
        causal_mask = ~causal_mask
        causal_mask = causal_mask.view(1, 1, seq_len, seq_len)
        causal_mask = causal_mask * attention_mask.view(1, 1, 1, seq_len)
        causal_mask = (causal_mask == 0).float() * torch.finfo(x.dtype).min
        
        with torch.no_grad():
            # Forcing layers_per_chunk to 5 as requested to test pipelining
            layers_per_chunk = 5
            x = compute(model, x, layers_per_chunk, cos, sin, k, tokens, causal_mask)
            
            x = model.model.norm(x)
            x = model.lm_head(x)
        
        # Reverting to the original greedy decoding using argmax as requested
        x1 = x[:,-1,:]
        next_token = torch.argmax(x1, dim=-1, keepdim=True)
        
        print(tok.decode(next_token[0]), end="")
        generated_text_tokens.append(tok.decode(next_token[0]))
        
        input_ids = torch.cat([input_ids, next_token.to('cuda')], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(attention_mask.shape[0], 1).to('cuda')], dim=-1)
    
    # Final cleanup at the very end
    model.model.embed_tokens.to('cpu')
    model.model.norm.to('cpu')
    model.lm_head.to('cpu')
    torch.cuda.empty_cache() 
    
    end_total_time = time.time()
    print(f"\nTime taken for {tokens} tokens: {end_total_time - start_total_time:.2f} seconds")
    return "".join(generated_text_tokens)




In [7]:

start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

birbirbirbirbirbirbirbirbirbir
Time taken for 10 tokens: 4.10 seconds

Total time 4.095994472503662


12.1
11.9
11.5
11.9
10.9
11.1
10.3
9.5
9.0 (10)
9.3 (11)
10.5 (12)
9.8 (13)
9.9 (14)
9.1 (15)

1.9 (16) 

In [217]:
question="hi how are "
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
prompt=tok(question,return_tensors='pt').to('cuda')
input_ids=prompt['input_ids']
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)

In [218]:
class module(nn.Module):
    def __init__(self,layers,i,j):
        super().__init__()
        self.model=nn.ModuleList(layers[i:j+1])
    def forward(self,x,cos,sin):
        for l in self.model:
            x=l(x,position_embeddings=(cos,sin))[0]
        return x

In [ ]:
import torch.nn as nn
class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    def forward(self, x, cos, sin):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x

def compute(model, x, layers_per_chunk, cos, sin):
    total_layers = len(model.model.layers)
    if layers_per_chunk >= total_layers:
        model = model.to('cuda')
        out = model({'inputs_embeds': x})['logits'][:, -1, :]
        return out.argmax(-1)
    i = 0
    rem = total_layers
    while rem > 0:
        l = min(layers_per_chunk, rem)
        m = Module(model.model.layers, i, i + l)

        # Run chunk
        x = m(x.to('cpu'), cos.to('cpu'), sin.to('cpu'))

        # update indices
        i += l
        rem -= l

    return x


In [211]:
y=compute(model,x,4,cos,sin)
y
model.model.norm.to('cpu')
x=model.model.norm(y)
model.model.norm.to('cpu')
torch.cuda.empty_cache()
model.lm_head.to('cpu')
x=model.lm_head(x)
# tok.decode(x)
x

tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4012,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>)

In [165]:
cos.device

device(type='cuda', index=0)

In [166]:
m=module(layers,0,7)
m(x.to('cpu'),cos.to('cpu'),sin.to('cpu'))

tensor([[[ 0.2278, -0.1952,  1.0512,  ...,  0.4470,  0.9027,  0.4017],
         [-0.0171,  0.0437,  0.0796,  ..., -0.0353, -0.0933, -0.0494],
         [-0.0076,  0.0305, -0.0894,  ..., -0.1101, -0.0858,  0.0215],
         [-0.0955,  0.1930, -0.0288,  ..., -0.0590, -0.0255,  0.1503],
         [-0.0718,  0.0508, -0.0462,  ...,  0.0025, -0.0664, -0.0365]]],
       grad_fn=<AddBackward0>)

In [226]:
question="hi how are "
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
prompt=tok(question,return_tensors='pt').to('cuda')
input_ids=prompt['input_ids']
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)

In [ ]:
class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    def forward(self, x, cos, sin):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x
def compute(model, x, layers_per_chunk, cos, sin):
    total_layers = len(model.model.layers)
    if layers_per_chunk >= total_layers:
        # model = model.to('cuda')
        # out = model({'inputs_embeds': x})['logits'][:, -1, :]
        # return out.argmax(-1)
        i=0
        l=total_layers
        m=Module(model.model.layers,i,i+l).to('cuda')
        x=m(x.to('cuda'),cos.to('cuda'),sin.to('cuda'))
        m=m.to('cpu')
        torch.cuda.empty_cache()
        return x
    i = 0
    rem = total_layers
    while rem > 0:
        l = min(layers_per_chunk, rem)
        m = Module(model.model.layers, i, i + l).to('cuda')

        # Run chunk
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'))
        m=m.to('cpu')
        torch.cuda.empty_cache()
        # update indices
        i += l
        rem -= l

    return x

In [230]:
y=compute(model,x,get_layers(model),cos,sin)
model.model.norm.to('cpu')
x=model.model.norm(y)
model.model.norm.to('cpu')
torch.cuda.empty_cache()
model.lm_head.to('cpu')
x=model.lm_head(x)
x

tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4012,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>)

In [225]:
model(input_ids.to('cpu'))

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4011,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x71d5247463e0>, hidden_states=None, attentions=None)

In [220]:
y=compute(model,x,4,cos,sin)
y

tensor([[[ 0.4072, -0.0109,  0.1410,  ...,  0.2553, -0.2387, -0.3508],
         [ 0.1307, -0.0781,  0.1387,  ...,  0.4531,  0.1098, -0.2780],
         [ 0.1409,  0.0029,  0.2776,  ...,  0.8693,  0.6018, -0.1494],
         [-0.0449,  0.0310,  0.0482,  ...,  0.3607, -0.3491, -0.5668],
         [-0.0326,  0.3774, -0.2075,  ...,  0.3641, -0.2429, -0.2585]]],
       grad_fn=<AddBackward0>)

In [221]:
model.model.norm.to('cpu')
x=model.model.norm(y)
model.model.norm.to('cpu')
torch.cuda.empty_cache()
model.lm_head.to('cpu')
x=model.lm_head(x)
# tok.decode(x)
x

tensor([[[ 0.4545,  2.3900, -1.2563,  ..., -1.2801, -1.2815, -1.2818],
         [-0.5335,  1.2898, -0.0679,  ..., -0.9111, -0.9123, -0.9122],
         [ 0.2582,  0.9092,  1.3728,  ..., -1.5733, -1.5758, -1.5756],
         [-1.0597,  2.9733,  0.6937,  ..., -0.5993, -0.6006, -0.6005],
         [ 0.2114,  0.6242,  1.3618,  ..., -0.0651, -0.0660, -0.0650]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
i=0
j=3
l=4
rem=16
while(rem!=0):
    if(rem>=l):
        m=nn.Sequential(
            layers[i:j]
        )
        print(i," ",j)
        rem-=l
        i+=l;j+=l
    else:
        m=nn.Sequential(
            layers[i:i+rem-1]
        )
        print('else')
        print(i," ",i+rem-1)
        rem=0


0   3
4   7
8   11
12   15


In [52]:
m=nn.Sequential(
        layers[0:3]
)
m

Sequential(
  (0): ModuleList(
    (0-2): 3 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
        (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
    )
  )
)

if int((torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)/param_size_MB)-2 > total layers  : load whole model into sequential model

else load int((torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)/param_size_MB)-2 layers into the seq 

keep track of the layers which went inside the seq model

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] if 6-2=4 layers are allowed then iterations = while i not>= number of layers 

In [ ]:
# LlamaRMSNorm((2048,), eps=1e-05)
#     (rotary_emb): LlamaRotaryEmbedding()
# print((sum(p.numel() for p in model.model.norm.parameters())*2)/(1024**3))
# print((sum(p.numel() for p in model.lm_head.parameters())*2)/(1024**3))
# print((sum(p.numel() for p in model.model.embed_tokens.parameters())*2)/(1024**3))


3.814697265625e-06
0.4892578125
0.4892578125


In [6]:
model=model.to('cpu')

In [7]:
torch.cuda.empty_cache()

In [8]:
question=input()
converstation=[
    {
        "role":"system",
        "content":"You are an assitant help user by solving their query"
    },
    {
        "role":"user",
        "content":f"query-{question}"
    }
]

In [57]:
question="hi how are "
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
# prompt=tok.apply_chat_template(converstation)
prompt=tok(question,return_tensors='pt').to('cuda')
# prompt.to('cpu'//)


In [58]:
prompt

{'input_ids': tensor([[128000,   6151,   1268,    527,    220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

In [37]:
# out=model.generate(**prompt,max_new_tokens=1000)
# print(tok.decode(out[0]))

In [63]:
prompt['input_ids'].device

device(type='cuda', index=0)

In [ ]:
class module(nn.Module):
    def __init__(self,layers,i,j):
        super().__init__()
        self.model=layers[i:j]
    def forward(self,x,cos,sin):
        for l in self.model:
            x=l(x,position_embeddings=(cos,sin))[0]
            print()
        return x

In [131]:
m=module(layers,0,3)
m(x.to('cpu'),cos.to('cpu'),sin.to('cpu'))

tensor([[[ 0.2181, -0.3543,  1.0459,  ...,  0.3998,  0.9639,  0.3820],
         [ 0.0501, -0.0546, -0.0243,  ...,  0.1164, -0.1678,  0.0384],
         [ 0.0353, -0.0942, -0.1205,  ..., -0.0287, -0.1063,  0.0035],
         [ 0.0649,  0.0549, -0.0413,  ..., -0.0517, -0.0086,  0.0737],
         [-0.0263,  0.0541, -0.0978,  ...,  0.0248, -0.0258, -0.0238]]],
       grad_fn=<AddBackward0>)

In [119]:
prompt['input_ids']

NameError: name 'prompt' is not defined

tensor([[[ 2.8381e-03,  3.3264e-03, -9.8877e-03,  ..., -1.7700e-03,
           8.4305e-04,  7.0190e-04],
         [-4.0039e-02,  2.8320e-02,  2.8076e-02,  ...,  1.8311e-02,
           9.5215e-03,  4.7363e-02],
         [-7.2937e-03,  4.3335e-03,  3.0029e-02,  ...,  8.0109e-05,
          -4.8828e-03,  1.2451e-02],
         [-1.6785e-03,  1.7334e-02,  3.9551e-02,  ..., -1.1047e-02,
          -1.5869e-02,  1.1520e-03],
         [ 5.0964e-03, -2.4902e-02,  3.6133e-02,  ..., -1.6602e-02,
          -1.8677e-02, -3.5156e-02]]], grad_fn=<ToCopyBackward0>)
tensor([[[ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
        

AttributeError: 'tuple' object has no attribute 'dtype'

In [107]:
m=model.model.layers[0]
m(x.to('cpu'),position_embeddings=(cos.to('cpu'),sin.to('cpu')))[0]

tensor([[[ 0.0131, -0.0472,  0.0412,  ..., -0.0090,  0.0760, -0.0241],
         [-0.0173, -0.0174, -0.0555,  ...,  0.0072, -0.0251, -0.0046],
         [-0.0093,  0.0029, -0.1033,  ..., -0.0247, -0.0518,  0.0019],
         [ 0.0227,  0.0492, -0.0206,  ..., -0.0301,  0.0070,  0.0488],
         [-0.0341,  0.0040, -0.0712,  ..., -0.0127, -0.0116,  0.0161]]],
       grad_fn=<AddBackward0>)

In [101]:
cos.device

device(type='cuda', index=0)

In [ ]:
# out=model(prompt['input_ids'].to('cpu'))
m=layers[:3]


ModuleList(
  (0-2): 3 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
      (k_proj): Linear(in_features=2048, out_features=512, bias=False)
      (v_proj): Linear(in_features=2048, out_features=512, bias=False)
      (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
      (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
      (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
    (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  )
)

In [106]:
x

tensor([[[ 2.8381e-03,  3.3264e-03, -9.8877e-03,  ..., -1.7700e-03,
           8.4305e-04,  7.0190e-04],
         [-4.0039e-02,  2.8320e-02,  2.8076e-02,  ...,  1.8311e-02,
           9.5215e-03,  4.7363e-02],
         [-7.2937e-03,  4.3335e-03,  3.0029e-02,  ...,  8.0109e-05,
          -4.8828e-03,  1.2451e-02],
         [-1.6785e-03,  1.7334e-02,  3.9551e-02,  ..., -1.1047e-02,
          -1.5869e-02,  1.1520e-03],
         [ 5.0964e-03, -2.4902e-02,  3.6133e-02,  ..., -1.6602e-02,
          -1.8677e-02, -3.5156e-02]]], device='cuda:0',
       grad_fn=<EmbeddingBackward0>)

In [85]:
model(prompt['input_ids'].to('cpu'))

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4011,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x71d526b7b8e0>, hidden_states=None, attentions=None)

In [76]:
nextt=out['logits'][:,-1,:].argmax(dim=-1)
nextt
tok.decode(nextt)

'2'

In [87]:
input_ids=prompt['input_ids']
del prompt
torch.cuda.empty_cache()

In [88]:
input_ids.device

device(type='cuda', index=0)

In [89]:
model.model.embed_tokens.to('cuda')
x = model.model.embed_tokens(input_ids)
input_ids.to('cpu')
# del input_ids
torch.cuda.empty_cache()
model.model.embed_tokens.to('cpu')
torch.cuda.empty_cache()


In [90]:
x

tensor([[[ 2.8381e-03,  3.3264e-03, -9.8877e-03,  ..., -1.7700e-03,
           8.4305e-04,  7.0190e-04],
         [-4.0039e-02,  2.8320e-02,  2.8076e-02,  ...,  1.8311e-02,
           9.5215e-03,  4.7363e-02],
         [-7.2937e-03,  4.3335e-03,  3.0029e-02,  ...,  8.0109e-05,
          -4.8828e-03,  1.2451e-02],
         [-1.6785e-03,  1.7334e-02,  3.9551e-02,  ..., -1.1047e-02,
          -1.5869e-02,  1.1520e-03],
         [ 5.0964e-03, -2.4902e-02,  3.6133e-02,  ..., -1.6602e-02,
          -1.8677e-02, -3.5156e-02]]], device='cuda:0',
       grad_fn=<EmbeddingBackward0>)

In [20]:
# model.model.decoder.embed_positions.to('cuda')
# x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
# model.model.decoder.embed_positions.to('cpu')
# # del model.model.decoder.embed_positions
# torch.cuda.empty_cache()

In [13]:
print(f"Model's max_position_embeddings: {model.config.max_position_embeddings}")

Model's max_position_embeddings: 131072


In [14]:
x.shape

torch.Size([1, 5, 2048])

In [91]:
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)

In [15]:
seq_len = x.shape[1]
batch_size = x.shape[0]
position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
rotary_emb = model.model.rotary_emb
cos, sin = rotary_emb(x=x,position_ids=position_ids)
cnt=0
for i in model.model.layers:
    cnt+=1
    print(cnt)
    i.to('cuda')
    x.to('cuda')
    # print(x)
    with torch.no_grad():
        x=i(x,position_embeddings=(cos,sin))
        # print(x)
    i.to('cpu')
    torch.cuda.empty_cache()
    x=x[0]
    # x.to('cpu')
    torch.cuda.empty_cache()
        # time.sleep(3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [16]:
x.shape

torch.Size([1, 5, 2048])

In [17]:
x

tensor([[[ 4.7762, -5.0166, 13.8534,  ..., -1.9152,  1.8989, -3.0755],
         [-0.2314,  1.3026,  1.7797,  ..., -0.2708, -1.0364, -0.2083],
         [ 0.2276,  0.8936,  0.8521,  ..., -0.0246, -1.4944, -0.0458],
         [ 0.0748,  1.2204,  0.5945,  ..., -0.7305, -1.4607, -0.6370],
         [ 0.6602,  1.3918, -0.1689,  ..., -1.1154, -1.7489, -0.1802]]],
       device='cuda:0')

In [18]:
x[:,-1,:]

tensor([[ 0.6602,  1.3918, -0.1689,  ..., -1.1154, -1.7489, -0.1802]],
       device='cuda:0')

In [19]:
with torch.no_grad():
    model.model.norm.to('cuda')
    x=model.model.norm(x)
    model.model.norm.to('cpu')
    torch.cuda.empty_cache()
    model.lm_head.to('cuda')
    x=model.lm_head(x)
    model.lm_head.to('cpu')
    torch.cuda.empty_cache()

In [48]:
x1.shape

AttributeError: 'list' object has no attribute 'shape'

In [20]:
x

tensor([[[ 7.0544,  9.0268, 13.3233,  ..., -3.7595, -3.7596, -3.7596],
         [17.6009,  9.2103,  9.4629,  ..., -0.6160, -0.6167, -0.6170],
         [12.9647,  9.7460,  8.3662,  ..., -0.2670, -0.2677, -0.2679],
         [14.1844, 10.6885,  6.5441,  ...,  0.4011,  0.4009,  0.4002],
         [ 7.1437,  4.0489,  5.2485,  ...,  1.8494,  1.8495,  1.8485]]],
       device='cuda:0')

In [60]:
prompt

NameError: name 'prompt' is not defined

In [24]:
torch.cat([input_ids,x1],dim=-1)

tensor([[128000,   6151,   1268,    527,    220,     17]], device='cuda:0')

In [22]:
x1=x[:,-1,:]
x1=torch.argmax(x1,dim=-1,keepdim=True)
print(tok.decode(x1[0]),end=" ")
# print(x1)

2 

In [8]:
# device=torch.device('cuda:0')
props=torch.cuda.get_device_properties('cuda')
props.total_memory/(1024**3)

7.6064453125

In [11]:
def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda')
    attention_mask=prompt['attention_mask']
    del prompt
    for i in range(tokens):
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        input_ids.to('cpu')
        # del input_ids
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cpu')
        torch.cuda.empty_cache()
        # Safe check
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        cnt=0
        for i in model.model.layers:
            cnt+=1
            # print(cnt)
            i.to('cuda')
            x.to('cuda')
            # print(x)
            with torch.no_grad():
                x=i(x,position_embeddings=(cos,sin))
                # print(x)
            i.to('cpu')
            torch.cuda.empty_cache()
            x=x[0]
            # x.to('cpu')
            torch.cuda.empty_cache()
            
        with torch.no_grad():
            model.model.norm.to('cuda')
            x=model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()
            model.lm_head.to('cuda')
            x=model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1) 
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
        

In [12]:
model_inference("Hi",10)

, I'm a new user and I'm trying

In [9]:
def model_inference(prompt_text, tokens=250):
    # Initial tokenization
    prompt = tok(prompt_text, return_tensors='pt')
    input_ids = prompt['input_ids'].to('cuda')  # [1, seq_len]
    del prompt
    torch.cuda.empty_cache()

    for _ in range(tokens):
        # Run only the latest token through embed layer and model
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        model.model.embed_tokens.to('cpu')
        torch.cuda.empty_cache()

        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)

        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x, position_ids=position_ids)

        # Transformer blocks
        for layer in model.model.layers:
            layer.to('cuda')
            x = x.to('cuda')
            with torch.no_grad():
                x = layer(x, position_embeddings=(cos, sin))
            layer.to('cpu')
            torch.cuda.empty_cache()
            x = x[0]  # Remove tuple

        # Final norm and lm_head
        with torch.no_grad():
            model.model.norm.to('cuda')
            x = model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()

            model.lm_head.to('cuda')
            logits = model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()

        next_token_logits = logits[:, -1, :]  # Only the last token matters
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)  # shape [1,1]

        # Append predicted token
        input_ids = torch.cat([input_ids, next_token], dim=-1)

        # Decode and print latest token
        print(tok.decode(next_token[0]), end="", flush=True)


In [11]:
model_inference("Hi how are you ?")

 I am very happy to meet you all. I am a very friendly person and I love to meet new people. I am a very good listener and I am very easy going. I am a very good cook and I love to cook. I am a very good dancer and I love to dance. I am a very good singer and I love to sing. I am a very good swimmer and I love to swim. I am a very good gardener and I love to garden. I am a very good cook and I love to cook. I am a very good dancer and I love to dance. I am a very good singer and I love to sing. I am a very good swimmer and I love to swim. I am a very good gardener and I love to garden. I am a very good cook and I love to cook. I am a very good dancer and I love to dance. I

KeyboardInterrupt: 

In [52]:
def tokenize_with_oov(text, tokenizer, max_token_id=2048, oov_token='[OOV]'):
    # First, add [OOV] token to tokenizer if not present
    if oov_token not in tokenizer.get_vocab():
        tokenizer.add_special_tokens({'additional_special_tokens': [oov_token]})

    oov_id = tokenizer.convert_tokens_to_ids(oov_token)

    # Raw tokenization
    encoding = tokenizer(text, return_tensors='pt', add_special_tokens=False)
    input_ids = encoding['input_ids'][0]

    # Replace tokens ≥ max_token_id with OOV
    processed_ids = []
    for token_id in input_ids:
        if token_id >= max_token_id:
            processed_ids.append(oov_id)
        else:
            processed_ids.append(token_id)

    return torch.tensor([processed_ids]), encoding['attention_mask']

def model_inference(question, tokens=250):
    try:
        for _ in range(tokens):
            # prompt = tok(question, return_tensors='pt').to('cpu')
            # input_ids = prompt['input_ids']
            # attention_mask = prompt['attention_mask']
            input_ids, attention_mask = tokenize_with_oov(question, tok)

            del prompt
            torch.cuda.empty_cache()

            # Truncate if too long
            batch_size, seq_len = input_ids.shape
            max_positions = model.config.max_position_embeddings
            if seq_len > max_positions:
                input_ids = input_ids[:, -max_positions:]
                attention_mask = attention_mask[:, -max_positions:]
                seq_len = input_ids.shape[1]

            # Embeddings
            model.model.decoder.embed_tokens.to('cpu')
            x = model.model.decoder.embed_tokens(input_ids)

            # Positional IDs
            position_ids = torch.arange(seq_len, dtype=torch.long, device='cpu')
            position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)

            model.model.decoder.embed_positions.to('cpu')
            x = x + model.model.decoder.embed_positions(position_ids)

            # Decoder layers
            for layer in model.model.decoder.layers:
                layer.to('cpu')
                x = x.to('cpu')
                with torch.no_grad():
                    x = layer(x, attention_mask=attention_mask.to(torch.float32))[0]
                layer.to('cpu')
                x = x.to('cpu')
                torch.cuda.empty_cache()

            # Final LM head
            model.lm_head.to('cpu')
            logits = model.lm_head(x.to('cpu'))
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()

            # Decode
            next_token_id = logits.argmax(-1)
            next_token_text = tok.decode(next_token_id[0], skip_special_tokens=True)
            print(next_token_text, end=" ")
            question += " " + next_token_text

    except Exception as e:
        print("\n[Exception Caught]")
        print("Question So Far:", question)
        prompt = tok(question, return_tensors='pt').to('cpu')
        input_ids = prompt['input_ids']
        x = model.model.decoder.embed_tokens(input_ids)
        y = torch.arange(x.shape[1]).unsqueeze(0)
        print("Embedding Shape:", x[0].shape)
        print("Position ID Shape:", y.shape)
        print("Error:", e)


In [56]:
model_inference("how are you 're post")

.''. you you'rewepost navigation .''. your you'reTheze is It.''..''.'re'reiresolicy姫conservancy how are you 're post .''. you you'rewepost navigation .''. your you'reTheze is It.''..''.'re'reiresolicy姫conservancy

torch.Size([35, 768])
torch.Size([1, 35])
index out of range in self


In [15]:
import torch
import gc

def model_inference(question, tokens=250):
    # Ensure model components are initially on CPU
    model.model.decoder.embed_tokens.to('cpu')
    model.model.decoder.embed_positions.to('cpu')
    for layer in model.model.decoder.layers:
        layer.to('cpu')
    model.lm_head.to('cpu')
    
    max_position_embeddings = model.config.max_position_embeddings
    print(f"Model's max_position_embeddings: {max_position_embeddings}")

    # Initialize input_ids_for_next_step. This will be the tensor that grows.
    input_ids_for_next_step = tok(question, return_tensors='pt')['input_ids'] 
    
    gc.collect()
    torch.cuda.empty_cache()

    for _ in range(tokens):
        # The 'x' variable for the current iteration is the input_ids_for_next_step
        current_input_ids = input_ids_for_next_step 
        current_seq_len = current_input_ids.shape[1]

        # --- Truncation ---
        if current_seq_len > max_position_embeddings:
            current_input_ids = current_input_ids[:, current_seq_len - max_position_embeddings:]
            print(f"Warning: Sequence truncated from {current_seq_len} to {current_input_ids.shape[1]} for max_position_embeddings.")
            current_seq_len = current_input_ids.shape[1] # Update after truncation

        # --- Embeddings ---
        model.model.decoder.embed_tokens.to('cpu')
        # Use current_input_ids for embedding lookup
        x_embeddings = model.model.decoder.embed_tokens(current_input_ids.to('cpu')) # x_embeddings is on CPU
        model.model.decoder.embed_tokens.to('cpu') # Move layer back
        gc.collect()
        
        # 2. Add Positional Embeddings
        model.model.decoder.embed_positions.to('cpu')
        try:
            position_offset = model.model.decoder.embed_positions.offset
        except AttributeError:
            position_offset = 2 # Common default for OPT

        pos_ids = torch.arange(position_offset, position_offset + current_seq_len, device='cpu').unsqueeze(0) 
        
        # The main `x` variable throughout the layer processing will be `current_token_embeddings_plus_pos`
        current_token_embeddings_plus_pos = x_embeddings + model.model.decoder.embed_positions(pos_ids) 
        model.model.decoder.embed_positions.to('cpu') 
        del x_embeddings 
        del pos_ids 
        gc.collect()

        # --- Decoder Layers Loop ---
        # Rename 'x' to something that reflects its current state (embeddings in, embeddings out)
        x_current_layer_output = current_token_embeddings_plus_pos # Start of layer processing
        for j, layer in enumerate(model.model.decoder.layers):
            try:
                layer.to('cpu')
                x_current_layer_output = x_current_layer_output.to('cpu') 

                with torch.no_grad():
                    x_current_layer_output = layer(x_current_layer_output)[0]

                layer.to('cpu')
                x_current_layer_output = x_current_layer_output.to('cpu') 
                
                gc.collect()

            except Exception as e:
                print(f"Error in decoder layer {j}: {e}")
                break
        else: 
            # --- LM Head and Token Generation ---
            model.lm_head.to('cpu')
            # The input to lm_head is x_current_layer_output (the final embeddings from decoder)
            logits = model.lm_head(x_current_layer_output.to('cpu')) 
            model.lm_head.to('cpu') 
            
            logits = logits.to('cpu') # Ensure logits are on CPU
            gc.collect()

            # Get the predicted token ID (on CPU)
            # Take the last token's prediction from batch 0
            predicted_token_id = logits.argmax(-1)[:, -1].item() 

            decoded_token = tok.decode(predicted_token_id)
            print(decoded_token, end=" ")
            
            # --- Prepare input_ids_for_next_step for the next iteration ---
            # Append the new token ID to the input_ids_for_next_step tensor (on CPU)
            new_token_tensor = torch.tensor([[predicted_token_id]], dtype=torch.long, device='cpu')
            
            # This is the correct concatenation for input_ids
            input_ids_for_next_step = torch.cat((input_ids_for_next_step, new_token_tensor), dim=1)
            
            # Update the 'question' string for printing and future reference
            question = question + " " + decoded_token
            
        if 'predicted_token_id' not in locals(): # If inner loop broke, stop outer loop
            break

In [4]:
def model_inference(question,tokens=250):
    for i in range(tokens):
        prompt=tok(question,return_tensors='pt').to('cuda')
        input_ids=prompt['input_ids']
        del prompt
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cuda')
        x = model.model.decoder.embed_tokens(input_ids)
        del input_ids
        torch.cuda.empty_cache()
        model.model.decoder.embed_tokens.to('cpu')
        # del model.model.decoder.embed_tokens
        torch.cuda.empty_cache()
        model.model.decoder.embed_positions.to('cuda')
        x = x + model.model.decoder.embed_positions(torch.arange(x.shape[1]).unsqueeze(0).to('cuda'))
        model.model.decoder.embed_positions.to('cpu')
        # del model.model.decoder.embed_positions
        torch.cuda.empty_cache()
        for i in model.model.decoder.layers:
            try:
                i.to('cuda')
                x=x.to('cuda')
                with torch.no_grad():
                    x=i(x)[0]
                i.to('cpu')
                torch.cuda.empty_cache()
                x=x.to('cpu')
                torch.cuda.empty_cache()
                # time.sleep(3)
            except:
                print(x)
                break
        model.lm_head.to('cuda')
        x=model.lm_head(x.to('cuda'))
        model.lm_head.to('cpu')
        torch.cuda.empty_cache()
        x1=x.argmax(-1)
        print(tok.decode(x1[0]),end=" ")
        question=question+" "+tok.decode(x1[0])
                

In [22]:
model_inference("How are you")

.''. your retarded're .''. you retarded're .dylib mom retard.''. 

NameError: name 'e' is not defined

In [11]:
print()
cnt=0
for i in model.model.decoder.layers:
    cnt+=1
    print(cnt)
    try:
        i.to('cuda')
        x.to('cuda')
        with torch.no_grad():
            x=i(x)[0]
        i.to('cpu')
        torch.cuda.empty_cache()
        x.to('cpu')
        torch.cuda.empty_cache()
        # time.sleep(3)
    except:
        print(cnt)
        print(x)
        break


1
2
3
4
5
6
7
8
9
10
11
12


In [12]:
model.lm_head.to('cuda')
x=model.lm_head(x.to('cuda'))
model.lm_head.to('cpu')
torch.cuda.empty_cache()

In [13]:
x1=x.argmax(-1)

In [14]:
tok.decode(x1[0])

' hi'

# 3 Workers

In [9]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc

# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )

model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           use_cache=True,
                                        #    quantization_config=bnb_config
                                           )

# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
       
    @torch.compile
    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x

layers=model.model.layers

def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda')
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu')
            torch.cuda.empty_cache()
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_previous = None

    # Load first chunk
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first

    while True:
        # Wait for transfer to complete before starting computation
        stream_compute.wait_stream(stream_transfer)
        
        # Move pointers
        m_previous = m_current
        m_current = m_next
        m_next = None

        # Compute on current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(x.to('cuda', non_blocking=True),
                              cos.to('cuda', non_blocking=True),
                              sin.to('cuda', non_blocking=True),
                              attention_mask.to('cuda', non_blocking=True))

        # Offload previous chunk to CPU (if exists)
        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)  # Wait for computation to finish before offloading
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
            torch.cuda.empty_cache()

        # If this is the last chunk, break
        if rem == 0:
            stream_compute.synchronize()
            # Offload the final chunk
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
            stream_offload.synchronize()
            torch.cuda.empty_cache()
            break

        # Load next chunk
        with torch.cuda.stream(stream_transfer):
            l_next = min(layers_per_chunk, rem)
            m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)

        i += l_next
        rem -= l_next

    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda')
    model.model.norm.to('cuda')
    model.lm_head.to('cuda')
    # x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb
   
    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cuda')
        # if i!=0:
        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
       
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
       
        cnt=0
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,2,cos,sin,k,tokens,attention_mask)
            # model.model.norm.to('cuda')
            x=model.model.norm(x)
            # model.model.norm.to('cpu')
            # torch.cuda.empty_cache()
            # model.lm_head.to('cuda')
            x=model.lm_head(x)
            # model.lm_head.to('cpu')
            # torch.cuda.empty_cache()
       
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1)
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
       



In [11]:
start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

, I am a 19 year old girl who
Total time 4.665896415710449


# 4 Workers 


In [1]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
from torch import nn
import time
import gc

# Add at the beginning of your script:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

bnb_config=BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_compute_dtype=torch.bfloat16,
            # bnb_4bit_use_double_quant=True,
            # bnb_8bit_quant_type='nf4',
        )

model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",
                                           torch_dtype=torch.bfloat16,
                                           use_cache=True,
                                          #  offload_folder='/offload_nvm',
                                          #  offload_state_dict=True,
                                          #  max_memory={"cpu":'0GB'},
                                        #    quantization_config=bnb_config
                                           )

# print(model)
tok=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id)

inputs = tokenizer("This is", return_tensors="pt").to("cuda")
kv_store = {}

def capture_kv(layer, inp, out):
    x = inp[0]
    attn = layer.self_attn
    kv_store[0] = {
        "k": attn.k_proj(x).detach().cpu(),
        "v": attn.v_proj(x).detach().cpu()
    }

layer = model.model.layers[0]

with torch.no_grad():
    x = model.model.embed_tokens(inputs["input_ids"])
    pos_ids = torch.arange(x.shape[1], device=x.device).unsqueeze(0)
    cos, sin = model.model.rotary_emb(x, pos_ids)

    capture_kv(layer, (x,), None)
    _ = layer(x, position_ids=pos_ids, position_embeddings=(cos, sin), use_cache=False)

for k, v in kv_store[0].items():
    print(f"{k}: {v.shape}")


/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


k: torch.Size([1, 3, 512])
v: torch.Size([1, 3, 512])


In [16]:

import sys
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory/param_size_GB)-drop)//2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
        # print(self.model.device)
       
    @torch.compile
    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            # print(layer.device)
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x

layers=model.model.layers
def slice_past(past_list, start, end):
    return past_list[start:end] if past_list else None

def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda',non_blocking=True)
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu',non_blocking=True)
            torch.cuda.empty_cache()
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_next2 = None
    m_previous = None

    # Load first two chunks
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first
        
        if rem > 0:
            l_second = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_second).to('cuda', non_blocking=True)
            i += l_second
            rem -= l_second
    while True:
    # Wait for previous transfer to complete before compute
        stream_compute.wait_stream(stream_transfer)

        # Move pointers forward
        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = None

        # Compute on current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(
                    x.to('cuda', non_blocking=True),
                    cos.to('cuda', non_blocking=True),
                    sin.to('cuda', non_blocking=True),
                    attention_mask.to('cuda', non_blocking=True)
                )

        # Offload the previous chunk after compute is done
        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
            torch.cuda.empty_cache()

        # If no more chunks left to load after this, finish
        if rem == 0 and m_next is None and m_next2 is None:
            # Wait for compute to complete
            stream_compute.synchronize()

            # Offload the current module
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
                del m_current
            stream_offload.synchronize()
            torch.cuda.empty_cache()
            break

        # Start preloading next-next chunk if any
        if rem > 0:
            l_next2 = min(layers_per_chunk, rem)
            stream_transfer2.wait_stream(stream_transfer)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
            i += l_next2
            rem -= l_next2
    return x
    # while True:
    #     # Wait for transfer to complete before starting computation
    #     stream_compute.wait_stream(stream_transfer)
        
    #     # Move pointers
    #     m_previous = m_current
    #     m_current = m_next
    #     m_next = m_next2
    #     m_next2 = None

    #     # Compute on current chunk
    #     with torch.cuda.stream(stream_compute):
    #         with torch.no_grad():
    #             x = m_current(x.to('cuda', non_blocking=True),
    #                           cos.to('cuda', non_blocking=True),
    #                           sin.to('cuda', non_blocking=True),
    #                           attention_mask.to('cuda', non_blocking=True))

    #     # Offload previous chunk to CPU (if exists)
    #     if m_previous is not None:
    #         stream_offload.wait_stream(stream_compute)  # Wait for computation to finish before offloading
    #         with torch.cuda.stream(stream_offload):
    #             m_previous = m_previous.to('cpu', non_blocking=True)
    #             del m_previous
    #         torch.cuda.empty_cache()

    #     # If this is the last chunk, break
    #     if rem == 0:
    #         stream_compute.synchronize()
    #         # Offload the final chunk
    #         with torch.cuda.stream(stream_offload):
    #             m_current = m_current.to('cpu', non_blocking=True)
    #         stream_offload.synchronize()
    #         torch.cuda.empty_cache()
    #         break

    #     # Load next chunk (2 steps ahead)
    #     # if rem > 0:
    #     #     time.sleep(0.01) 
    #     #     l_next2 = min(layers_per_chunk, rem)
    #     #     with torch.cuda.stream(stream_transfer2):
    #     #         m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
    #     #     time.sleep(0.1)
    #     #     i += l_next2
    #     #     rem -= l_next2
    #     if rem > 0:
    #         l_next2 = min(layers_per_chunk, rem)

    #         # Create CUDA events for synchronization
    #         transfer_start = torch.cuda.Event(blocking=False)
    #         transfer_end = torch.cuda.Event(blocking=True)  # blocking=True ensures CPU waits for it

    #         # Record the start of the transfer on the stream
    #         stream_transfer2.record_event(transfer_start)

    #         with torch.cuda.stream(stream_transfer2):
    #             m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
    #             stream_transfer2.record_event(transfer_end)  # Record when transfer is done

    #         # Wait until transfer is completed before proceeding
    #         transfer_end.synchronize()  # 100% guaranteed sync

    #         # Advance the layer index
    #         i += l_next2
    #         rem -= l_next2

    # return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb
   
    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cuda',non_blocking=True)
        # if i!=0:
        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
       
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
       
        cnt=0
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,2,cos,sin,k,tokens,attention_mask)
            # model.model.norm.to('cuda',non_blocking=True)
            x=model.model.norm(x)
            # model.model.norm.to('cpu',non_blocking=True)
            torch.cuda.empty_cache()
            # model.lm_head.to('cuda',non_blocking=True)
            x=model.lm_head(x)
            # model.lm_head.to('cpu',non_blocking=True)
            torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1)
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
    input_ids.to('cpu')
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
       



In [20]:
start=time.time()
model_inference("HI how are you",50)
print()
print("Total time",time.time()-start)

? I'm a big fan of your work. I'm a big fan of your work. I'm a big fan of your work. I'm a big fan of your work. I'm a big fan of your work. I'm a big
Total time 18.601060390472412


In [3]:
def get_layers(model, drop=2):
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024 ** 3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory / param_size_GB) - drop) // 2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    @torch.compile
    def forward(self, x, cos, sin, attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x

layers = model.model.layers

def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()

    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        time.sleep(5.01)  # delay 1
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'))
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
            torch.cuda.empty_cache()
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_next2 = None
    m_previous = None

    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        time.sleep(5.01)  # delay 2
        i += l_first
        rem -= l_first

        if rem > 0:
            l_second = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_second).to('cuda', non_blocking=True)
            time.sleep(5.01)  # delay 3
            i += l_second
            rem -= l_second

    while True:
        stream_compute.wait_stream(stream_transfer)
        time.sleep(5.01)  # delay 4

        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = None

        time.sleep(5.01)  # delay 5

        with torch.cuda.stream(stream_compute):
            time.sleep(5.01)  # delay 6
            with torch.no_grad():
                x = m_current(x.to('cuda', non_blocking=True),
                              cos.to('cuda', non_blocking=True),
                              sin.to('cuda', non_blocking=True),
                              attention_mask.to('cuda', non_blocking=True))

        time.sleep(5.01)  # delay 7

        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            time.sleep(5.01)  # delay 8
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
            time.sleep(5.01)  # delay 9
            torch.cuda.empty_cache()

        if rem == 0:
            stream_compute.synchronize()
            time.sleep(8)  # delay 10
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
            stream_offload.synchronize()
            time.sleep(8)  # delay 11
            torch.cuda.empty_cache()
            break

        if rem > 0:
            print("runing test")
            time.sleep(15)  # delay 12
            l_next2 = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
            time.sleep(15)  # delay 13
            i += l_next2
            rem -= l_next2

    return x

def model_inference(question, tokens=250):
    prompt = tok(question, return_tensors='pt').to('cpu', non_blocking=True)
    input_ids = prompt['input_ids'].to('cuda', non_blocking=True)
    attention_mask = prompt['attention_mask']
    model.model.embed_tokens.to('cuda', non_blocking=True)
    model.model.norm.to('cuda', non_blocking=True)
    model.lm_head.to('cuda', non_blocking=True)
    time.sleep(5.01)  # delay 14

    x = model.model.embed_tokens(input_ids)
    rotary_emb = model.model.rotary_emb

    del prompt
    for k in range(tokens):
        torch.cuda.empty_cache()
        time.sleep(5.01)  # delay 15

        x = model.model.embed_tokens(input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)

        cos, sin = rotary_emb(x=x, position_ids=position_ids)
        time.sleep(5.01)  # delay 16

        with torch.no_grad():
            l1 = get_layers(model, 7)
            x = compute(model, x, 9, cos, sin, k, tokens, attention_mask)
            time.sleep(5.01)  # delay 17

            x = model.model.norm(x)
            time.sleep(5.01)  # delay 18

            x = model.lm_head(x)
            time.sleep(5.01)  # delay 19

        x1 = x[:, -1, :]
        x1 = torch.argmax(x1, dim=-1, keepdim=True)
        print(tok.decode(x1[0]), end="")
        input_ids = torch.cat([input_ids, x1], dim=-1)
        time.sleep(5.01)  # delay 20

    input_ids.to('cpu')
    model.model.norm.to('cpu', non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu', non_blocking=True)
    torch.cuda.empty_cache()


In [4]:
start=time.time()
model_inference("HI",10)
print()
print("Total time",time.time()-start)

W0805 11:36:46.319881 52923 site-packages/torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:2177: UserWarning: NVIDIA T1000 8GB does not support bfloat16 compilation natively, skipping
  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/torch/_inductor/lowering.py:7007: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/pa/miniconda3/envs/gen_ai/lib/pyt

HI nonetheless

KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import nn
import time

torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

def get_layers(model, drop=2):
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024**3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory / param_size_GB) - drop) // 2

class KVCache:
    def __init__(self, num_layers):
        self.num_layers = num_layers
        # Store KV cache on CPU to save GPU memory
        self.cache = {}
        
    def get_cache(self, layer_idx):
        """Get cache for a specific layer"""
        return self.cache.get(layer_idx, None)
    
    def update_cache(self, layer_idx, k, v):
        """Update cache for a specific layer"""
        if layer_idx not in self.cache:
            self.cache[layer_idx] = {"k": k.detach().cpu(), "v": v.detach().cpu()}
        else:
            # Concatenate new k,v with existing cache along sequence dimension
            existing_k = self.cache[layer_idx]["k"]
            existing_v = self.cache[layer_idx]["v"]
            self.cache[layer_idx]["k"] = torch.cat([existing_k, k.detach().cpu()], dim=2)  # Fixed: dim=2 for seq_len
            self.cache[layer_idx]["v"] = torch.cat([existing_v, v.detach().cpu()], dim=2)  # Fixed: dim=2 for seq_len
    
    def clear(self):
        """Clear all cache"""
        self.cache.clear()

def apply_rotary_pos_emb(q, k, cos, sin, position_ids):
    """Apply rotary positional embeddings to query and key tensors"""
    batch_size = q.shape[0]
    q_seq_len = q.shape[2]
    k_seq_len = k.shape[2]
    
    # Get cos/sin for query positions
    if position_ids.shape[1] == q_seq_len:
        # Use provided position_ids directly
        cos_q = cos[:, position_ids.squeeze(0), :].unsqueeze(1)  # [batch_size, 1, seq_len, head_dim]
        sin_q = sin[:, position_ids.squeeze(0), :].unsqueeze(1)
    else:
        # Single token case
        pos_idx = position_ids.squeeze()
        cos_q = cos[:, pos_idx:pos_idx+1, :].unsqueeze(1)
        sin_q = sin[:, pos_idx:pos_idx+1, :].unsqueeze(1)
    
    # Apply to query
    q_embed = (q * cos_q) + (rotate_half(q) * sin_q)
    
    # For keys, handle full sequence
    if k_seq_len == q_seq_len:
        # Same sequence length, use same cos/sin
        cos_k = cos_q
        sin_k = sin_q
    else:
        # Different sequence length (cached keys), create full position sequence
        start_pos = k_seq_len - q_seq_len
        full_positions = torch.arange(start_pos, k_seq_len, device=k.device, dtype=torch.long)
        cos_k = cos[:, full_positions, :].unsqueeze(1)
        sin_k = sin[:, full_positions, :].unsqueeze(1)
    
    k_embed = (k * cos_k) + (rotate_half(k) * sin_k)
    
    return q_embed, k_embed

def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def capture_and_apply_kv(layer, x, layer_idx, kv_cache, position_embeddings, attention_mask, position_ids, is_first_token=False):
    """Capture K,V values and apply KV caching for a layer"""
    cos, sin = position_embeddings
    
    # PRE-NORM: Apply input layer norm first
    hidden_states = layer.input_layernorm(x)
    
    # Get attention module
    attn = layer.self_attn
    
    # Model dimensions for Llama-3.2-1B
    num_heads = 32
    head_dim = 64
    num_key_value_heads = 8
    
    # Compute Q, K, V projections
    q = attn.q_proj(hidden_states)
    k_new = attn.k_proj(hidden_states)
    v_new = attn.v_proj(hidden_states)
    
    bsz, seq_len, _ = q.shape
    
    # Reshape for multi-head attention
    q = q.view(bsz, seq_len, num_heads, head_dim).transpose(1, 2)
    k_new = k_new.view(bsz, seq_len, num_key_value_heads, head_dim).transpose(1, 2)
    v_new = v_new.view(bsz, seq_len, num_key_value_heads, head_dim).transpose(1, 2)
    
    # Handle KV caching - fixed to avoid position embedding issues
    if is_first_token:
        # First token - no existing cache, apply RoPE to all tokens
        k_full = k_new
        v_full = v_new
        kv_cache.update_cache(layer_idx, k_new, v_new)
        
        # Apply rotary embeddings to q and k
        if cos is not None and sin is not None:
            q, k_full = apply_rotary_pos_emb(q, k_full, cos, sin, position_ids)
    else:
        # Get existing cache and concatenate
        cached = kv_cache.get_cache(layer_idx)
        if cached is not None:
            k_cached = cached["k"].to(k_new.device)
            v_cached = cached["v"].to(v_new.device)
            
            # Apply RoPE to new k tokens only
            if cos is not None and sin is not None:
                q, k_new = apply_rotary_pos_emb(q, k_new, cos, sin, position_ids)
            
            # Concatenate cached (already has RoPE) with new (just applied RoPE)
            k_full = torch.cat([k_cached, k_new], dim=2)
            v_full = torch.cat([v_cached, v_new], dim=2)
            
            # Update cache with the new tokens
            kv_cache.update_cache(layer_idx, k_new, v_new)
        else:
            # No cache, treat as first token
            k_full = k_new
            v_full = v_new
            kv_cache.update_cache(layer_idx, k_new, v_new)
            if cos is not None and sin is not None:
                q, k_full = apply_rotary_pos_emb(q, k_full, cos, sin, position_ids)
    
    # Expand k,v for grouped query attention (GQA: 8 KV heads -> 32 Q heads)
    if num_key_value_heads != num_heads:
        k_full = k_full.repeat_interleave(num_heads // num_key_value_heads, dim=1)
        v_full = v_full.repeat_interleave(num_heads // num_key_value_heads, dim=1)
    
    # Compute attention scores
    attn_weights = torch.matmul(q, k_full.transpose(2, 3)) / torch.sqrt(torch.tensor(head_dim, dtype=q.dtype, device=q.device))
    full_seq_len = k_full.shape[2]
    # Apply causal mask - only mask future tokens
    if full_seq_len > seq_len:
        # We have cached tokens, create appropriate mask
        causal_mask = torch.tril(torch.ones(seq_len, full_seq_len, device=q.device, dtype=torch.bool))
    else:
        causal_mask = torch.tril(torch.ones(seq_len, seq_len, device=q.device, dtype=torch.bool))
    
    # Apply the mask
    attn_weights = attn_weights.masked_fill(~causal_mask.unsqueeze(0).unsqueeze(0), float('-inf'))
    
    # Apply softmax
    attn_weights = torch.softmax(attn_weights, dim=-1, dtype=torch.float32).to(q.dtype)
    
    # Apply attention to values
    attn_output = torch.matmul(attn_weights, v_full)
    
    # Reshape back to [batch_size, seq_len, hidden_size]
    attn_output = attn_output.transpose(1, 2).contiguous().view(bsz, seq_len, -1)
    
    # Apply output projection
    attn_output = attn.o_proj(attn_output)
    
    # Add residual connection
    hidden_states = x + attn_output
    
    # POST-NORM: Apply post attention layer norm
    residual = hidden_states
    hidden_states = layer.post_attention_layernorm(hidden_states)
    
    # Apply MLP
    mlp_output = layer.mlp(hidden_states)
    
    # Add residual connection
    hidden_states = residual + mlp_output
    
    return hidden_states

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
        self.start_idx = i
    
    def forward(self, x, cos, sin, attention_mask, kv_cache, position_ids, is_first_token=False):
        for idx, layer in enumerate(self.model):
            layer_idx = self.start_idx + idx
            x = capture_and_apply_kv(layer, x, layer_idx, kv_cache, (cos, sin), attention_mask, position_ids, is_first_token)
        return x

def compute(model_instance, x, layers_per_chunk, cos, sin, k, max_new_tokens, attention_mask, kv_cache, position_ids, is_first_token=False):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'), attention_mask.to('cuda'), kv_cache, position_ids.to('cuda'), is_first_token)
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
            torch.cuda.empty_cache()
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_next2 = None
    m_previous = None

    # Load first two chunks
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first
        
        if rem > 0:
            l_second = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_second).to('cuda', non_blocking=True)
            i += l_second
            rem -= l_second

    while True:
        # Wait for previous transfer to complete before compute
        stream_compute.wait_stream(stream_transfer)

        # Move pointers forward
        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = None

        # Compute on current chunk
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(
                    x.to('cuda', non_blocking=True),
                    cos.to('cuda', non_blocking=True),
                    sin.to('cuda', non_blocking=True),
                    attention_mask.to('cuda', non_blocking=True),
                    kv_cache,
                    position_ids.to('cuda', non_blocking=True),
                    is_first_token
                )

        # Offload the previous chunk after compute is done
        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
            torch.cuda.empty_cache()

        # If no more chunks left to load after this, finish
        if rem == 0 and m_next is None and m_next2 is None:
            # Wait for compute to complete
            stream_compute.synchronize()

            # Offload the current module
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
                del m_current
            stream_offload.synchronize()
            torch.cuda.empty_cache()
            break

        # Start preloading next-next chunk if any
        if rem > 0:
            l_next2 = min(layers_per_chunk, rem)
            stream_transfer2.wait_stream(stream_transfer)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
            i += l_next2
            rem -= l_next2
    return x

def model_inference(question, tokens=250):
    # Add proper prompt formatting for Llama
    if not question.strip():
        question = "Hello! How are you today?"
    
    prompt = tok(question, return_tensors='pt', add_special_tokens=True)
    input_ids = prompt['input_ids'].to('cuda', non_blocking=True)
    attention_mask = prompt['attention_mask'].to('cuda', non_blocking=True)
    
    print(f"Input: {question}")
    print("Output: ", end="")
    
    # Initialize KV cache
    num_layers = len(model.model.layers)
    kv_cache = KVCache(num_layers)
    
    model.model.embed_tokens.to('cuda', non_blocking=True)
    model.model.norm.to('cuda', non_blocking=True)
    model.lm_head.to('cuda', non_blocking=True)
    
    rotary_emb = model.model.rotary_emb
    
    for k in range(tokens):
        torch.cuda.empty_cache()
        
        # Determine if this is the first token generation
        is_first_token = (k == 0)
        
        # For subsequent tokens, only process the last token
        if not is_first_token:
            current_input_ids = input_ids[:, -1:].contiguous()
        else:
            current_input_ids = input_ids
            
        x = model.model.embed_tokens(current_input_ids)
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        
        # Position IDs for current token(s)
        if is_first_token:
            position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        else:
            # For subsequent tokens, position ID should be the current sequence length
            current_pos = input_ids.shape[1] - 1
            position_ids = torch.tensor([[current_pos]], dtype=torch.long, device=x.device)
        
        # Get rotary embeddings - simplified approach
        max_seq_len = 4096  # Use a reasonable max length
        dummy_x = torch.zeros(batch_size, max_seq_len, x.shape[-1], device=x.device, dtype=x.dtype)
        dummy_position_ids = torch.arange(max_seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        cos, sin = rotary_emb(x=dummy_x, position_ids=dummy_position_ids)
        
        with torch.no_grad():
            l1 = get_layers(model, 7)
            x = compute(model, x, 3, cos, sin, k, tokens, attention_mask, kv_cache, position_ids, is_first_token)
            x = model.model.norm(x)
            torch.cuda.empty_cache()
            x = model.lm_head(x)
            torch.cuda.empty_cache()
        
        # Get the logits for the last token
        logits = x[:, -1, :]
        
        # Use temperature and top-p sampling for better generation
        temperature = 0.7
        top_p = 0.9
        
        # Apply temperature
        logits = logits / temperature
        
        # Apply top-p filtering
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        
        indices_to_remove = sorted_indices_to_remove.scatter(dim=-1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = float('-inf')
        
        # Sample from the filtered distribution
        probs = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        # Check for end of sequence
        if next_token.item() == tok.eos_token_id:
            break
            
        print(tok.decode(next_token[0], skip_special_tokens=True), end="", flush=True)
        input_ids = torch.cat([input_ids, next_token], dim=-1)
    
    print()  # New line at the end
    
    # Cleanup
    input_ids.to('cpu')
    model.model.embed_tokens.to('cpu', non_blocking=True)
    model.model.norm.to('cpu', non_blocking=True)
    model.lm_head.to('cpu', non_blocking=True)
    torch.cuda.empty_cache()
    kv_cache.clear()

# Load model and tokenizer
tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", torch_dtype=torch.bfloat16, use_cache=True)
layers = model.model.layers

# Test the model
while True:
    question = input("\nEnter your question (or 'quit' to exit): ")
    if question.lower() == 'quit':
        break
    start = time.time()
    model_inference(question, 50)  # Reduced tokens for testing
    print(f"Time taken: {time.time() - start:.2f}s")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...
Model loaded successfully!
Input: Hello! How are you today?
Output: 

AttributeError: 'LlamaAttention' object has no attribute 'num_key_value_heads'

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import nn
import time
import math

# Use bfloat16 for better performance on modern GPUs
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

def get_layers(model, drop=2):
    """
    Estimate the number of layers that can fit on the GPU.
    This is a heuristic and may need adjustment.
    """
    device_memory = (torch.cuda.get_device_properties('cuda').total_memory) / (1024**3)
    l = model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2  # Assuming bfloat16 (2 bytes per parameter)
    param_size_GB = param_size_bytes / (1024 ** 3)
    
    # Ensure at least one layer can fit and we don't return zero
    return max(1, (int(device_memory / param_size_GB) - drop) // 2)

class KVCache:
    """A simple KV cache implementation on the CPU to save GPU memory."""
    def __init__(self, num_layers, num_key_value_heads, head_dim, max_seq_len=4096):
        self.num_layers = num_layers
        self.num_key_value_heads = num_key_value_heads
        self.head_dim = head_dim
        self.max_seq_len = max_seq_len
        self.cache = {}
    
    def get_cache(self, layer_idx):
        """Get cache for a specific layer."""
        return self.cache.get(layer_idx, None)
    
    def update_cache(self, layer_idx, k, v, seq_pos):
        """Update cache for a specific layer at specific sequence position."""
        if layer_idx not in self.cache:
            batch_size = k.shape[0]
            self.cache[layer_idx] = {
                "k": torch.zeros(batch_size, self.num_key_value_heads, self.max_seq_len, self.head_dim, 
                                 dtype=k.dtype, device='cpu'),
                "v": torch.zeros(batch_size, self.num_key_value_heads, self.max_seq_len, self.head_dim, 
                                 dtype=v.dtype, device='cpu'),
                "seq_len": 0
            }
        
        cache_entry = self.cache[layer_idx]
        current_seq_len = k.shape[2]
        
        cache_entry["k"][:, :, seq_pos : seq_pos + current_seq_len, :] = k.detach().cpu()
        cache_entry["v"][:, :, seq_pos : seq_pos + current_seq_len, :] = v.detach().cpu()
        cache_entry["seq_len"] = seq_pos + current_seq_len
    
    def get_cached_kv(self, layer_idx, device):
        """Get cached k,v up to current sequence length."""
        if layer_idx not in self.cache:
            return None, None, 0
            
        cache_entry = self.cache[layer_idx]
        seq_len = cache_entry["seq_len"]
        
        if seq_len == 0:
            return None, None, 0
            
        k_cached = cache_entry["k"][:, :, :seq_len, :].to(device)
        v_cached = cache_entry["v"][:, :, :seq_len, :].to(device)
        
        return k_cached, v_cached, seq_len
    
    def clear(self):
        """Clear all cache."""
        self.cache.clear()

def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

# Removed apply_rotary_pos_emb as it will be handled directly by the rotary_emb module

def llama_attention_forward(layer, hidden_states, rotary_emb, position_ids, kv_cache, layer_idx, past_seq_len=0):
    """Custom forward pass for Llama attention with KV caching"""
    attn = layer.self_attn
    
    bsz, q_len, _ = hidden_states.shape
    
    # Hardcoded values based on Llama-3.2-1B model
    num_heads = 32
    num_key_value_heads = 8
    head_dim = 64
    
    # Project to Q, K, V
    query_states = attn.q_proj(hidden_states)
    key_states = attn.k_proj(hidden_states)
    value_states = attn.v_proj(hidden_states)
    
    # Reshape for multi-head attention
    query_states = query_states.view(bsz, q_len, num_heads, head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, num_key_value_heads, head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, num_key_value_heads, head_dim).transpose(1, 2)
    
    # Apply rotary position embeddings using the passed rotary_emb module
    query_states, key_states = rotary_emb(query_states, key_states, position_ids)
    
    # Handle KV caching
    cached_k, cached_v, cache_seq_len = kv_cache.get_cached_kv(layer_idx, key_states.device)
    
    if cached_k is not None:
        key_states = torch.cat([cached_k, key_states], dim=2)
        value_states = torch.cat([cached_v, value_states], dim=2)
    
    kv_cache.update_cache(layer_idx, key_states[:, :, past_seq_len:, :], value_states[:, :, past_seq_len:, :], past_seq_len)
    
    # Repeat k/v heads if num_key_value_heads < num_heads (grouped query attention)
    if num_key_value_heads != num_heads:
        key_states = key_states.repeat_interleave(num_heads // num_key_value_heads, dim=1)
        value_states = value_states.repeat_interleave(num_heads // num_key_value_heads, dim=1)
    
    # Compute attention
    attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(head_dim)
    
    # Apply causal mask
    seq_len_q = query_states.shape[2]
    seq_len_k = key_states.shape[2]
    
    causal_mask = torch.full((seq_len_q, seq_len_k), float('-inf'), device=hidden_states.device, dtype=attn_weights.dtype)
    causal_mask = torch.triu(causal_mask, diagonal=past_seq_len + 1)
        
    attn_weights = attn_weights + causal_mask.unsqueeze(0).unsqueeze(0)
    
    # Apply softmax
    attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
    
    # Apply attention to values
    attn_output = torch.matmul(attn_weights, value_states)
    
    # Reshape and apply output projection
    attn_output = attn_output.transpose(1, 2).contiguous().view(bsz, q_len, -1)
    attn_output = attn.o_proj(attn_output)
    
    return attn_output

def llama_layer_forward(layer, hidden_states, rotary_emb, position_ids, kv_cache, layer_idx, past_seq_len=0):
    """Custom forward pass for a Llama layer"""
    residual = hidden_states
    hidden_states = layer.input_layernorm(hidden_states)
    attn_output = llama_attention_forward(layer, hidden_states, rotary_emb, position_ids, kv_cache, layer_idx, past_seq_len)
    hidden_states = residual + attn_output
    residual = hidden_states
    hidden_states = layer.post_attention_layernorm(hidden_states)
    mlp_output = layer.mlp(hidden_states)
    hidden_states = residual + mlp_output
    return hidden_states

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
        self.start_idx = i
    
    def forward(self, x, rotary_emb, position_ids, kv_cache, past_seq_len=0):
        for idx, layer in enumerate(self.model):
            layer_idx = self.start_idx + idx
            x = llama_layer_forward(layer, x, rotary_emb, position_ids, kv_cache, layer_idx, past_seq_len)
        return x

def compute(model_instance, x, layers_per_chunk, rotary_emb, position_ids, kv_cache, past_seq_len, k, max_new_tokens):
    total_layers = len(model_instance.model.layers)
    if layers_per_chunk >= total_layers:
        m = Module(model_instance.model.layers, 0, total_layers).to('cuda', non_blocking=True)
        x = m(x.to('cuda'), rotary_emb.to('cuda'), position_ids.to('cuda'), kv_cache, past_seq_len)
        if k == max_new_tokens - 1:
            m.to('cpu', non_blocking=True)
            torch.cuda.empty_cache()
        return x

    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()
    
    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_previous = None

    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first

    while True:
        stream_compute.wait_stream(stream_transfer)
        m_previous = m_current
        m_current = m_next
        m_next = None

        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(
                    x.to('cuda', non_blocking=True),
                    rotary_emb.to('cuda', non_blocking=True),
                    position_ids.to('cuda', non_blocking=True),
                    kv_cache,
                    past_seq_len
                )

        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_previous.to('cpu', non_blocking=True)
                del m_previous
            torch.cuda.empty_cache()

        if rem == 0:
            stream_compute.synchronize()
            with torch.cuda.stream(stream_offload):
                m_current.to('cpu', non_blocking=True)
                del m_current
            stream_offload.synchronize()
            torch.cuda.empty_cache()
            break

        if rem > 0:
            l_next = min(layers_per_chunk, rem)
            with torch.cuda.stream(stream_transfer):
                m_next = Module(model_instance.model.layers, i, i + l_next).to('cuda', non_blocking=True)
            i += l_next
            rem -= l_next
    return x

def model_inference(question, max_new_tokens=50):
    if not question.strip():
        question = "hi"
    
    inputs = tok(question, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids'].to('cuda')
    
    print(f"Input: {question}")
    print("Output: ", end="", flush=True)
    
    num_layers = len(model.model.layers)
    
    # Hardcoded values for Llama-3.2-1B
    num_key_value_heads = 8
    head_dim = 64
    
    # Create the KVCache with the correct hardcoded parameters
    kv_cache = KVCache(
        num_layers=num_layers,
        num_key_value_heads=num_key_value_heads,
        head_dim=head_dim,
        max_seq_len=4096
    )
    
    # Get the rotary_emb module from the main model.model object
    # This is the correct way to access it, as it's a shared component
    rotary_emb = model.model.rotary_emb.to('cuda', non_blocking=True)
    
    model.model.embed_tokens.to('cuda', non_blocking=True)
    model.model.norm.to('cuda', non_blocking=True)
    model.lm_head.to('cuda', non_blocking=True)
    
    generated_tokens = []
    
    for k in range(max_new_tokens):
        torch.cuda.empty_cache()
        
        is_first_iteration = (k == 0)
        
        if is_first_iteration:
            current_input_ids = input_ids
            past_seq_len = 0
        else:
            current_input_ids = input_ids[:, -1:].contiguous()
            past_seq_len = input_ids.shape[1] - 1
        
        hidden_states = model.model.embed_tokens(current_input_ids)
        
        seq_len = hidden_states.shape[1]
        position_ids = torch.arange(past_seq_len, past_seq_len + seq_len, dtype=torch.long, device=hidden_states.device)
        position_ids = position_ids.unsqueeze(0).expand(hidden_states.shape[0], -1)
        
        with torch.no_grad():
            layers_per_chunk = get_layers(model, 7)
            hidden_states = compute(
                model, hidden_states, layers_per_chunk, rotary_emb, position_ids, 
                kv_cache, past_seq_len, k, max_new_tokens
            )
            
            hidden_states = model.model.norm(hidden_states)
            torch.cuda.empty_cache()
            
            logits = model.lm_head(hidden_states)
            torch.cuda.empty_cache()
        
        next_token_logits = logits[:, -1, :]
        
        temperature = 0.7
        top_p = 0.9
        
        if temperature > 0:
            next_token_logits = next_token_logits / temperature
            
            sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True)
            cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            
            indices_to_remove = sorted_indices_to_remove.scatter(dim=-1, index=sorted_indices, src=sorted_indices_to_remove)
            next_token_logits[indices_to_remove] = float('-inf')
            
            probs = torch.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
        else:
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
        
        if next_token.item() == tok.eos_token_id:
            break
        
        token_text = tok.decode(next_token[0], skip_special_tokens=True)
        print(token_text, end="", flush=True)
        generated_tokens.append(next_token.item())
        
        input_ids = torch.cat([input_ids, next_token], dim=-1)
    
    print()
    
    model.model.embed_tokens.to('cpu', non_blocking=True)
    model.model.norm.to('cpu', non_blocking=True)
    model.lm_head.to('cpu', non_blocking=True)
    rotary_emb.to('cpu', non_blocking=True) # Move rotary_emb back to CPU
    torch.cuda.empty_cache()
    kv_cache.clear()

# Load model and tokenizer
print("Loading model...")
tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B", 
    torch_dtype=torch.bfloat16, 
    device_map=None
)

print("Model loaded successfully!")

# Test the model
question = "hi"
start = time.time()
model_inference(question, 50)
print(f"Time taken: {time.time() - start:.2f}s")

/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...
Model loaded successfully!
Input: hi
Output: 

TypeError: LlamaRotaryEmbedding.forward() takes 3 positional arguments but 4 were given

In [1]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import nn

# Enable TF32 on supported cards for faster matmuls
torch.backends.cuda.matmul.allow_tf32 = True

def get_layers_per_chunk(model, drop_gb=2):
    total_mem_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    # Estimate size of one layer in GB
    example_layer = model.model.layers[0]
    num_params = sum(p.numel() for p in example_layer.parameters())
    # Map dtype to bytes per element
    dtype_size = torch.tensor([], dtype=model.dtype).element_size()
    layer_size_gb = num_params * dtype_size / 1024**3
    return max(1, int((total_mem_gb - drop_gb) // layer_size_gb))

class KVCache:
    def __init__(self, num_layers):
        self.cache = [None] * num_layers

    def get(self, idx):
        return self.cache[idx]

    def update(self, idx, present):
        if self.cache[idx] is None:
            self.cache[idx] = present
        else:
            k_old, v_old = self.cache[idx]
            k_new, v_new = present
            # concat on seq dim (-2)
            self.cache[idx] = (torch.cat([k_old, k_new], dim=-2), torch.cat([v_old, v_new], dim=-2))

    def clear(self):
        self.cache = [None] * len(self.cache)

class LayerChunk(nn.Module):
    def __init__(self, layers, start, end):
        super().__init__()
        self.layers = nn.ModuleList(layers[start:end])
        self.start = start

    def forward(self, x, attn_mask, cos, sin, kv_cache, first_token):
        for i, layer in enumerate(self.layers):
            idx = self.start + i
            past = kv_cache.get(idx)
            out, present = layer(
                x,
                position_embeddings=(cos, sin),
                past_key_value=past,
                use_cache=True
            )
            x = out
            kv_cache.update(idx, present)
        return x


def compute_stream(model, tokenizer, input_ids, max_new_tokens=10):
    device = torch.device('cuda')
    batch, seq_len = input_ids.shape

    # Prepare cache and move critical parts
    num_layers = len(model.model.layers)
    kv_cache = KVCache(num_layers)
    model.model.embed_tokens.to(device)
    model.model.norm.to(device)
    model.lm_head.to(device)

    rot_emb = model.model.rotary_emb
    chunks = get_layers_per_chunk(model)

    x = None
    for step in range(max_new_tokens):
        is_first = (step == 0)
        cur_ids = input_ids if is_first else input_ids[:, -1:]
        x = model.model.embed_tokens(cur_ids.to(device))

                # Rotary embeddings
        cur_seq_len = cur_ids.shape[1]
        if is_first:
            pos_ids = torch.arange(cur_seq_len, device=device).unsqueeze(0).expand(batch, -1)
        else:
            pos_ids = torch.tensor([[seq_len-1]], device=device).expand(batch, 1)
        cos, sin = rot_emb(x=x, position_ids=pos_ids)

        # Process in chunks
        rem = num_layers
        start = 0
        attn_mask = torch.ones_like(cur_ids, device=device)
        while rem > 0:
            end = start + chunks
            chunk = LayerChunk(model.model.layers, start, min(end, num_layers)).to(device)
            x = chunk(x, attn_mask, cos, sin, kv_cache, is_first)
            # offload
            chunk.to('cpu')
            torch.cuda.empty_cache()
            start += chunks
            rem -= chunks

        x = model.model.norm(x)
        logits = model.lm_head(x)
        nxt = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        print(tokenizer.decode(nxt[0]), end='', flush=True)
        input_ids = torch.cat([input_ids.to(device), nxt], dim=-1)
        seq_len += 1

    print()
    # Cleanup
    model.model.embed_tokens.to('cpu')
    model.model.norm.to('cpu')
    model.lm_head.to('cpu')
    kv_cache.clear()


def main():
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-3.2-1B",
        torch_dtype=torch.bfloat16,
        use_cache=False,
        low_cpu_mem_usage=True
    )
    model.eval()

    # Single-shot prompt = "hi"
    prompt = "hi"
    inputs = tokenizer(prompt, return_tensors='pt')
    print(f"Prompt: {prompt}")
    print("Output:", end=' ')
    start = time.time()
    compute_stream(model, tokenizer, inputs.input_ids, max_new_tokens=10)
    print(f"Time: {time.time() - start:.2f}s")

if __name__ == "__main__":
    main()


/home/pa/miniconda3/envs/gen_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Prompt: hi
Output: 

ValueError: not enough values to unpack (expected 2, got 1)